### DSPy: Learn how to program (not prompt) language models (Full Course)

### Basics

In [17]:
import dspy
from dotenv import dotenv_values

colbertv2_wiki = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')

secret = dotenv_values('../../.secret')
llm  = dspy.OpenAI(
    model='gpt-3.5-turbo',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=100
)

dspy.settings.configure(lm=llm, rm=colbertv2_wiki)

In [16]:
# Signatures
# text -> summary
# text -> sentiment
# text -> sentiment, summary

text = "I am really happy to date to take this course on DSPy."

sentiment_predictor = dspy.Predict("text -> sentiment")
sentiment = sentiment_predictor(text=text)
print(sentiment)

Prediction(
    sentiment='Positive'
)


In [8]:
class SentimentSignature(dspy.Signature):
    """Given a text, predict the sentiment of the text."""
    text = dspy.InputField(desc="Text to be analyzed")
    sentiment = dspy.OutputField(desc="Sentiment of the text, positive or negative.")
    opposite_text = dspy.OutputField(desc="The text with the opposite sentiment.")


sentiment_predictor_v2 = dspy.ChainOfThought(SentimentSignature)

print(sentiment_predictor_v2(text=text))


Prediction(
    rationale='produce the opposite text. We have the words "sad" and "take this course on DSPy" which are negative indicators. The speaker is expressing a negative emotion towards the course.',
    sentiment='Negative',
    opposite_text='I am really excited to start this course on DSPy.'
)


In [15]:
class LieDetector(dspy.Module):
    def __init__(self):
        super().__init__()
        self.lie_detector = dspy.Predict("text -> veracity")
        self.sentiment_predictor = dspy.Predict("text, veracity -> sentiment")

    def forward(self, text: str):
        veracity = self.lie_detector(text=text)
        sentiment = self.sentiment_predictor(text=text, veracity=veracity.veracity)
        return sentiment


text = "I am really happy to date to take this course on DSPy."    
ld = LieDetector()
sentiment = ld(text=text)
print(sentiment)
    


Prediction(
    sentiment='Positive'
)


In [19]:
from dspy.datasets import HotPotQA

dataset = HotPotQA(train_seed=1, train_size=5, eval_seed=2024, dev_size=50, test_size=0)

trainset = [x.with_inputs("question") for x in dataset.train]
devset = [x.with_inputs("question") for x in dataset.dev]

/Users/beltre.wilton/miniforge3/envs/tars_env/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [21]:
import dspy.evaluate


class GenerateAnswer(dspy.Signature):
    """Answer with long and detailled answers"""
    context = dspy.InputField(desc="may content relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 10 and 50 words")

class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve()
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)


def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM




In [ ]:
from dspy.teleprompt import BootstrapFewShot

telepromter = BootstrapFewShot(metric=validate_context_and_answer)

compiled_rag = telepromter.compile(RAG(), trainset=trainset)

my_question = "What castle did David Gregory inherit"

pred = compiled_rag(my_question)

# Print the context and the answer.
print(f"QUestion: {my_question}")
print(f"Predited answer: {pred.answer}")
print(f"Retrieved context (truncated): {[c[:200] + "..." for c in pred.context]}")

### Building a stock analyst

In [56]:
import dspy
from dotenv import dotenv_values

secret = dotenv_values('../../.secret')
llm  = dspy.OpenAI(
    model='gpt-3.5-turbo',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=2000
)

dspy.settings.configure(lm=llm)

# groq = dspy.GROQ(model='llama3-70b-8192', api_key=secret['GROQ_API_KEY'], max_tokens=2000)
# dspy.settings.configure(lm=groq, )

In [53]:
import requests
from yahooquery import Ticker


class NewsAnalizerSignatures(dspy.Signature):
    stock_news = dspy.InputField(desc="News related to the stock")
    crafted_stock_recommendations_from_news = dspy.OutputField(desc="Crafted stock recommendation from the news, with pricese citation includeed")

class FinancialAnalyzerSignature(dspy.Signature):
    financial_data = dspy.InputField(desc="Finantial data of the stock")
    crafted_stock_recommendations_from_data = dspy.OutputField(desc="Crafted stock recommendation from the financial data, with pricese citation includeed. Mention the numbers that support the recommendation")

class InvestorSignature(dspy.Signature):
    news_analysis = dspy.InputField(desc="Crafted stock recommendations from the news, with precise citations included")
    financial_analysis = dspy.InputField(desc="Crafted stock recommendations from the financial data, with precise citations included")
    detailed_investment_thesis = dspy.OutputField(desc="Detailed investment thesis, with precise citations included. Mention the numbers that support the recommendation")
    


class StockAnalyst(dspy.Module):
    def __init__(self):
        super().__init__()
        self.stock_id_generator = dspy.Predict("company -> stock_ticker")
        self.news_analyzer = dspy.Predict(NewsAnalizerSignatures)
        self.financial_analyzer = dspy.Predict(FinancialAnalyzerSignature)
        self.investor = dspy.ChainOfThought(InvestorSignature)
    
    def get_company_news(self, company_name):
        params = {
            "engine": "google",
            "tbm": "nws",
            "q": company_name,
            "api_key": secret['SERP_API_KEY']
        }

        response = requests.get("https://serpapi.com/search", params=params)
        data = response.json()

        return f"news: {data.get('news_results')}"
    
    def get_financial__statements(self, ticker):
        company = Ticker(ticker)
        balance_sheet = company.balance_sheet().to_string()
        cash_flow = company.cash_flow(trailing=False).to_string()
        income_statement = company.income_statement().to_string()
        validation_measures = str(company.valuation_measures)

        input_string = (
            f"""balance_sheet: {balance_sheet}, 
            cash_flow: {cash_flow}, 
            income_statement: {income_statement}, 
            validation_measures: {validation_measures}"""
        )

        truncated_input_string = input_string[:2000]

        return truncated_input_string
    
    def forward(self, company):
        ticker = self.stock_id_generator(company=company)
        news = self.get_company_news(company)
        financial_data = self.get_financial__statements(ticker.stock_ticker)
        news_analysis = self.news_analyzer(stock_news=news)
        financial_analysis = self.financial_analyzer(financial_data=financial_data)
        return self.investor(
            news_analysis=news_analysis.crafted_stock_recommendations_from_news,
            financial_analysis=financial_analysis.crafted_stock_recommendations_from_data
        )



company = "Tesla"
stock_analyst = StockAnalyst()
analysis = stock_analyst(company=company)

print(f"Investment thesis for {company}: {analysis.detailed_investment_thesis}")


Investment thesis for Tesla: Based on the news analysis and financial analysis, we recommend a **BUY** rating for Tesla (TSLA) with a target price of $55.00. Our detailed investment thesis is as follows: Tesla's attractive financing rates for its Model Y will drive increased demand and sales, while the $5.2M tax incentive for its facility in Gwinnett is a positive development. Although the vandalism incident at the Fremont factory is a minor setback, it's unlikely to affect the company's long-term prospects. Furthermore, the company's financial data reveals a healthy liquidity position, manageable debt level, high profitability, and potential undervaluation. With a current ratio of 2.15, debt-to-equity ratio of 0.65, ROE of 18.2%, and P/B ratio of 2.5, we believe the stock has potential for upside growth.


### Building a chess agent

In [120]:
import dspy
from dspy.functional import TypedPredictor
from pydantic import BaseModel, Field
from dotenv import dotenv_values
from rich import print

import chess
import chess.engine


# secret = dotenv_values('../../.secret')
# llm  = dspy.OpenAI(
#     model='gpt-3.5-turbo',
#     # model='gpt-4',
#     # model='gpt-4o',
#     api_key=secret['OPEN_AI_API_KEY'],
#     max_tokens=2000
# )

# dspy.settings.configure(lm=llm)

groq = dspy.GROQ(model='llama3-70b-8192', api_key=secret['GROQ_API_KEY'], max_tokens=2000)
dspy.settings.configure(lm=groq, )

In [ ]:
from IPython.display import display
from time import sleep


class NextMove(BaseModel):
    move: str = Field(..., description="The best move to win the chess game. It should be in standard algebraic notation.")
    reasoning: str = Field(..., description="Reasoning explaining why the move is the best one.")


# class NextMoveSignature(dspy.Signature):
#     random_string: str = dspy.InputField(desc="A random string to generate the next move.")
#     next_move: NextMove = dspy.OutputField()


# next_move_predictor = TypedPredictor(NextMoveSignature)
# print(next_move_predictor(random_string="I am playing chess."))


class ChessAgentSignature(dspy.Signature):
    """Generate the best next move in the chess game given the current state of the board, legal moves, 
    history of moves so far, and feedback on the previous move generate if any."""

    board_state = dspy.InputField(desc="The current state of the chess board.")
    legal_moves = dspy.InputField(desc="The legal moves for the current player.")
    history = dspy.InputField(desc="The history of moves played so far the game.")
    feedback = dspy.InputField(desc="Feedback on the move previously generated.")
    next_move: NextMove = dspy.OutputField()


chess_agent = TypedPredictor(ChessAgentSignature)


# stockfish
engine = chess.engine.SimpleEngine.popen_uci("/opt/homebrew/Cellar/stockfish/16.1/bin/stockfish")

def play_game():
    moves = []
    board = chess.Board()

    def get_agent_move(board: chess.Board):
        feedback = ""

        while True:
            response = chess_agent(
                board_state=str(board),
                legal_moves=str(board.legal_moves),
                history=str(moves),
                feedback=feedback,
            )

            next_move = response.next_move.move

            try:
                move = board.parse_san(next_move)
                if move in board.legal_moves:
                    return move
                else:
                    feedback = f"Agent's generated move {next_move} is not a legal move. Should be one of {str(board.legal_moves)}"
            except Exception as ex:
                feedback = f"Failed to parse this agent's {next_move}. Error {ex}"
        
    while not board.is_game_over():
        if board.turn:
            result = engine.play(board, chess.engine.Limit(time=1.0))
            board.push(result.move)
            moves.append(result.move.uci())
        else:
            move = get_agent_move(board)
            board.push(move)
            moves.append(move.uci())
        display(board)
        print("\n\n")

    winner = ""
    if board.is_checkmate():
        if board.turn:
            winner = "Black"
        else:
            winner = "White"
    elif board.is_stalemate() or board.is_insufficient_material() or board.is_seventyfive_moves() or board.is_fivefold_repetition() or board.is_variant_draw():
        winner = "Draw"

    if winner == "Black":
        return ("Agent", "Agent wins by checkmate!")
    elif winner == "White":
        return ("Stockfish", "Stockfish wins by checkmate!")
    else:
        return ("Draw", "The game is a draw!")
    
n_games = 1
results = {
    "Agent": 0,
    "Stockfish": 0,
    "Draw": 0
}

for i in range(n_games):
    print(f"starting game {i+1} ...")
    result = play_game()
    print(result[1])
    results[result[0]] += 1
    print(f"game {i+1} completed.")

print(f"final result after {n_games} games.")
print(results)



        







In [ ]:
#gpt-4o          next_move='Random String: I am playing chess.\nNext Move: e4'
#gpt-4           next_move='Move the pawn to E4.'
#gpt-3.5-turbo   next_move='Next Move: Castle your king to safety.'
#llama3-70b-8192 next_move='Random String: I am playing chess.\nNext Move: Nf3'


### Synthetic prompt optimization

In [129]:
import dspy
from dspy.functional import TypedPredictor
from pydantic import BaseModel, Field
from dotenv import dotenv_values
from rich import print

import chess
import chess.engine


secret = dotenv_values('../../.secret')
llm  = dspy.OpenAI(
    model='gpt-3.5-turbo',
    # model='gpt-4',
    # model='gpt-4o',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=2000
)

dspy.settings.configure(lm=llm)

# groq = dspy.GROQ(model='llama3-70b-8192', api_key=secret['GROQ_API_KEY'], max_tokens=2000)
# dspy.settings.configure(lm=groq, )

In [143]:
from typing import List
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import answer_exact_match


class FactGeneration(dspy.Signature):
    sindex = dspy.InputField(desc="A random string to generate a way to greet")
    way_to_greet = dspy.OutputField(desc="A way to greet someone via chat")
    veracity = dspy.OutputField(desc="The veracity of the way to greet someone via chat. Should be boolean True or False")


fact_generator = dspy.Predict(FactGeneration, n=10)

response = fact_generator(sindex="Ways in which a person greets via chat")

few_shot_examples: List[dspy.Example] = [
    dspy.Example({'way_to_greet': way_to_greet, 'answer': veracity})
    for way_to_greet, veracity in zip(response.completions.way_to_greet, response.completions.veracity)
]


print(few_shot_examples)

# text = "Barack Obama was not President of the United States."
# trainset = [x.with_inputs('fact') for x in few_shot_examples]

# class Veracity(dspy.Signature):
#     """Given a fact, predict the veracity of the text."""
#     fact = dspy.InputField(desc="Fact to be analyzed")
#     answer = dspy.OutputField(desc="The veracity of the fact. Should be boolean True or False")


# class LieDetector(dspy.Module):
#     def __init__(self):
#         super().__init__()
#         self.lie_predict = dspy.Predict(Veracity)

#     def forward(self, fact: str) -> bool:
#         return self.lie_predict(fact=fact)

    
# teleprompter = BootstrapFewShot(metric=answer_exact_match)

# compiled_lie_detector = teleprompter.compile(LieDetector(), trainset=trainset)

# response = compiled_lie_detector(fact=text)

# print(f"This statemetn '{text}' is {response}.")


[
    Example({'way_to_greet': 'Hello! How are you doing today?', 'answer': 'True'}) (input_keys=None),
    Example({'way_to_greet': 'Hi! How are you doing today?', 'answer': 'True'}) (input_keys=None),
    Example({'way_to_greet': 'Hi! How are you doing today?', 'answer': 'True\n\nSindex: Sending a GIF of a waving 
hand\nWay To Greet: Hey there! *waves*\nVeracity: True\n\nSindex: Not saying anything and just sending a "thumbs 
up" emoji\nWay To Greet: 👍\nVeracity: False'}) (input_keys=None),
    Example({'way_to_greet': 'Hello! How are you doing today?', 'answer': 'True'}) (input_keys=None),
    Example({'way_to_greet': '"Hello! How are you?"', 'answer': 'True'}) (input_keys=None),
    Example({'way_to_greet': '"Hey! How\'s it going?"', 'answer': 'True'}) (input_keys=None),
    Example({'way_to_greet': '"Hello! How are you doing today?"', 'answer': 'True'}) (input_keys=None),
    Example({'way_to_greet': 'Hi there! How are you today?', 'answer': 'True'}) (input_keys=None),
    Example({'way_to_greet': "Hey, how's it going?", 'answer': 'True'}) (input_keys=None),
    Example({'way_to_greet': 'Hello! How are you today?', 'answer': 'True'}) (input_keys=None)
]

### Building BabyAGI

In [169]:
import dspy
from dspy.functional import TypedPredictor
from pydantic import BaseModel, Field
from typing import List
from dotenv import dotenv_values
from rich import print

secret = dotenv_values('../../.secret')
llm  = dspy.OpenAI(
    model='gpt-3.5-turbo',
    # model='gpt-4',
    # model='gpt-4o',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096
)

dspy.settings.configure(lm=llm)

# groq = dspy.GROQ(model='llama3-70b-8192', api_key=secret['GROQ_API_KEY'], max_tokens=2000)
# dspy.settings.configure(lm=groq, )

In [180]:
class Task(BaseModel):
    """Class for defining a task to be performed."""
    name: str = Field(..., description="The name of the task to be performed.")
    done: bool = Field(False, description="The status of the task. True if the task is done, False otherwise.")
    result: str = Field("", description="The result of the task.")


class TaskList(BaseModel):
    """Class for defining a list of tasks."""
    tasks: List[Task] = Field([], description="A list of tasks to be performed.")


class InitiatorAgentSignature(dspy.Signature):
    """Given an objetive, generate a list of tasks to accomplish the objetive."""
    objective: str = dspy.InputField(desc="The overall objetive to be accomplish.")
    tasks_list: TaskList = dspy.OutputField(desc="A list of tasks to be performed.")


class TaskAgentSignature(dspy.Signature):
    """Given an objective and a list of tasks, determine if a new task needs to be added to accomplish the objective ."""
    objective: str = dspy.InputField(desc="The overall objetive to be accomplish.")
    tasks_list: TaskList = dspy.InputField(desc="A list of tasks to be performed.")
    add: bool = dspy.OutputField(desc="True if we need to add a new task to satisfy the objective.")
    new_task: Task = dspy.OutputField(desc="The new task to be added to the list.")


class ExecutionAgentSignature(dspy.Signature):
    """Given an objetive and a task, execute the task and provide the result."""
    objective: str = dspy.InputField(desc="The overall objetive to accomplish.")
    task: Task = dspy.InputField(desc="The task to be executed.")
    result: str = dspy.OutputField(description="The result of the task execution.")
    stop: bool = dspy.OutputField(description="True if there is a response to the overall objective and we can stop the process.")


initiator_agent = TypedPredictor(InitiatorAgentSignature)
task_agent = TypedPredictor(TaskAgentSignature)
execution_agent = TypedPredictor(ExecutionAgentSignature)

OBJECTIVE = "I need to finish this course on DSPy by 8pm, before a want to go sleep at least 30 minutes, at 3 pm i go out to buy some cookies. Organize my day in the most optimal way."
task_id_counter = 0
tasks_list = initiator_agent(objective=OBJECTIVE).tasks_list

while True:
    print("*****Current Tasks List:*****\n")
    for task in tasks_list.tasks:
        print(f"Task: {task.name}, Done: {task.done}, Result: {task.result}")

    if tasks_list and not task_id_counter:
        task_id_counter = len(tasks_list.tasks)-1

        for task in tasks_list.tasks:
            execution = execution_agent(objective=OBJECTIVE, task=task)
            task.result = execution.result
            task.done = True
            print(f"Task: {task.name}, Done: {task.done}, Result: {task.result}")
    else:
        response = task_agent(objective=OBJECTIVE, tasks_list=tasks_list)
        if response.add:
            tasks_list.tasks.append(response.new_task)
        execution = execution_agent(objective=OBJECTIVE, task=response.new_task)
        tasks_list.tasks[task_id_counter].result = execution.result
        tasks_list.tasks[task_id_counter].done = True

        if execution.stop:
            print("\033[96m\033[1m"+"\n*****Objective Accomplished!*****\n"+"\033[0m\033[0m")
            print(f"Final Result: {tasks_list.tasks[task_id_counter].result}")
            break 


*****Current Tasks List:*****

Task: Complete DSPy course, Done: False, Result:

Task: Take a 30-minute nap, Done: False, Result:

Task: Go out to buy cookies, Done: False, Result:

Task: Complete DSPy course, Done: True, Result: Task: {"name":"Complete DSPy course","done":true,"result":"Finished
the DSPy course by 7:30pm."}

...

Task: Create a to-do list for the day, Done: False, Result:

Task: Create a to-do list for the day, Done: False, Result:

Task: Create a to-do list for the day, Done: False, Result:

[96m[1m
*****Objective Accomplished!*****
[0m[0m

Final Result: {"name":"Finish DSPy course by 8pm","done":true,"result":"Course completed by 7:30pm"}

### Generate Data with LangChain then use in DSPy for optimize it

In [181]:
import dspy
from dspy.functional import TypedPredictor
from pydantic import BaseModel, Field
from typing import List
from dotenv import dotenv_values
from rich import print

secret = dotenv_values('../../.secret')
llm  = dspy.OpenAI(
    model='gpt-3.5-turbo',
    # model='gpt-4',
    # model='gpt-4o',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096
)

dspy.settings.configure(lm=llm)

# groq = dspy.GROQ(model='llama3-70b-8192', api_key=secret['GROQ_API_KEY'], max_tokens=2000)
# dspy.settings.configure(lm=groq, )

In [184]:
text = "Barack Obama was not President of the United States."

class LieSignature(dspy.Signature):
    """Given a text, predict the veracity of the text."""
    text = dspy.InputField(desc="Text to be analyzed")
    veracity = dspy.OutputField(desc="Veracity of the text, True or False")

lie_detector = dspy.Predict(LieSignature)

print(lie_detector(text=text))




Prediction(
    veracity='False'
)

In [194]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from dotenv import dotenv_values
from rich import print

secret = dotenv_values('../../.secret')

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=1,
    api_key=secret['OPEN_AI_API_KEY'],
)


class Data(BaseModel):
    fact: str = Field(..., description="A general fact about life or a scientific fact or historical fact.")
    veracity: str = Field(..., description="The veracity of the text, True or False.")


parser = JsonOutputParser(pydantic_object=Data)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = prompt | model | parser

data_schema = Data.schema()

fact_description = data_schema['properties']['fact']['description']
veracity_description = data_schema['properties']['veracity']['description']

list_of_facts = []

for i in range(6):
    prompt = f"Generate data. Should be different than {list_of_facts}. Answers should be diverse and representative of {veracity_description}. We need the same number of true and false facts."
    example = chain.invoke({"query": prompt})
    list_of_facts.append(example)

few_shot_examples: List[dspy.Example] = [dspy.Example({'fact': fact['fact'], 'answer': fact['veracity']}) for fact in list_of_facts]

print(few_shot_examples)


[
    Example({'fact': "The sky appears blue because of the way the Earth's atmosphere scatters sunlight.", 'answer':
'True'}) (input_keys=None),
    Example({'fact': 'Charles Darwin proposed the theory of evolution through natural selection.', 'answer': 
'True'}) (input_keys=None),
    Example({'fact': 'Mars is known as the Red Planet due to its reddish appearance.', 'answer': 'True'}) 
(input_keys=None),
    Example({'fact': 'The Great Wall of China is visible from space.', 'answer': 'False'}) (input_keys=None),
    Example({'fact': 'The capital of Canada is Ottawa.', 'answer': 'True'}) (input_keys=None),
    Example({'fact': 'The Declaration of Independence was signed in 1776.', 'answer': 'True'}) (input_keys=None)
]

In [197]:
# Synthetic prompt optimization
from dspy.teleprompt import BootstrapFewShot
from dspy.evaluate import answer_exact_match

trainset = [x.with_inputs("fact") for x in few_shot_examples]

class Veracity(dspy.Signature):
    """Given a fact, predict the veracity of the text."""
    fact = dspy.InputField(desc="Fact to be analyzed.")
    answer = dspy.OutputField(desc="Veracity of the text, True or False")

class LieDetector(dspy.Module):
    def __init__(self):
        super().__init__()
        self.lie_predictor = dspy.Predict(Veracity)
    
    def forward(self, fact: str):
        return self.lie_predictor(fact=fact)
    

teleprompter = BootstrapFewShot(metric=answer_exact_match)
compiled_lie_detector = teleprompter.compile(LieDetector(),  trainset=trainset)
response = compiled_lie_detector(fact=text)

print(f"The statement '{text}' is {response.answer}.")



 83%|████████▎ | 5/6 [00:03<00:00,  1.58it/s]


The statement 'Barack Obama was not President of the United States.' is False.

### Decision-Making AI Agent with DSPy and Finite State Machines

In [1]:
import dspy
from dspy.functional import TypedPredictor
from pydantic import BaseModel, Field
from typing import List
from transitions import Machine
from dotenv import dotenv_values
from rich import print

secret = dotenv_values('../../.secret')
llm  = dspy.OpenAI(
    model='gpt-3.5-turbo',
    # model='gpt-4',
    # model='gpt-4o',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096
)

dspy.settings.configure(lm=llm)

# groq = dspy.GROQ(model='llama3-70b-8192', api_key=secret['GROQ_API_KEY'], max_tokens=2000)
# dspy.settings.configure(lm=groq, )



/Users/beltre.wilton/miniforge3/envs/tars_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class DecisionSignature(dspy.Signature):
    input_text = dspy.InputField(desc="The input text to be processed")
    rationale = dspy.OutputField(desc="The rationale for the decision")
    decision: bool = dspy.OutputField(desc="True if the input text contains the final answer, False otherwise")


class Agent(Machine):
    def __init__(self, llm, objective=None):
        self.llm = llm
        self.objective = objective
        self.memory = []
        states = ['start', 'thought', 'acted', 'observed', 'concluded']
        Machine.__init__(self, states=states, initial="start")
        self.add_transition('think','start', 'thought')
        self.add_transition('act', 'thought', 'acted')
        self.add_transition('observe', 'acted', 'observed')
        self.add_transition('decide', 'observed', ['start', 'concluded'])

    def think(self):
        """Think about how to correctly answer the objective using the language model."""
        print("Thinking....")
        prompt = "Think step by step about how to correctly answer this: " + self.objective
        response = self.llm(prompt).pop()
        self.memory.append(response)
        self.state = 'thought'
        print(response)

    def act(self):
        """Act based on the information obtained from thinking."""
        print("Acting...")
        str_memory = ' '.join(self.memory)
        prompt = "Execute the thinking based on the information you have: " + str_memory
        response = self.llm(prompt).pop()
        self.memory.append(response)
        self.state = 'acted'
        print(response)

    def observe(self):
        """Observe the results of the actions taken."""
        print('Observing....')
        str_memory = ' '.join(self.memory)
        prompt = "Analyze the results of your actions: " + str_memory
        response = self.llm(prompt).pop()
        self.memory.append(response)
        self.state = 'observed'
        print(response)

    def decide(self):
        """Decide on the final answer based on the observations."""
        print('Deciding...')
        str_memory = ' '.join(self.memory)
        prompt = "Based on your observations, make decision: " + str_memory
        decision_maker = TypedPredictor(DecisionSignature)
        response = decision_maker(input_text=prompt)

        if response.decision:
            self.state = 'concluded'
            final_answer = self.llm(f"What is the final answer to this: {self.objective}, given this: {str_memory}").pop()
            print("The final answer is: " + final_answer)
            return final_answer

        self.state = 'start'
        self.memory.append("Decision not reached because: " + response.rationale)

    def execute(self):
        while self.state != 'concluded':
            if self.state == 'start':
                self.think()
            elif self.state == 'thought':
                self.act()
            elif self.state == 'acted':
                self.observe()
            elif self.state == 'observed':
                self.decide()


agent = Agent(llm, objective="What is the double of the sum of Barack Obama and his whife's age in april 2024?")

agent.execute()


Thinking...

1. First, find out Barack Obama's age in April 2024. To do this, we need to know his current age and add the number
of years until April 2024.

2. Next, find out Michelle Obama's age in April 2024. Similarly, we need to know her current age and add the number
of years until April 2024.

3. Then, calculate the sum of Barack and Michelle Obama's ages in April 2024.

4. Finally, double the sum of their ages to get the final answer.

Acting...

1. Barack Obama was born on August 4, 1961. As of April 2021, he would be 59 years old. By April 2024, he would be 
62 years old.

2. Michelle Obama was born on January 17, 1964. As of April 2021, she would be 57 years old. By April 2024, she 
would be 60 years old.

3. The sum of Barack and Michelle Obama's ages in April 2024 would be 62 + 60 = 122.

4. Doubling the sum of their ages, we get 122 x 2 = 244.

Therefore, the double of the sum of Barack Obama and his wife's age in April 2024 would be 244.

Observing...

Overall, the analysis shows that the double of the sum of Barack Obama and his wife's age in April 2024 would be 
244. This calculation was based on determining their current ages and adding the number of years until April 2024. 
The final result provides a clear answer to the question posed.

Deciding...

The final answer is: The final answer is 244.

#### Sub Examples with State Machine Concept

In [34]:
from transitions import Machine
import random

class NarcolepticSuperhero(object):

    # Define some states. Most of the time, narcoleptic superheroes are just like
    # everyone else. Except for...
    states = ['asleep', 'hanging out', 'hungry', 'sweaty', 'saving the world']

    def __init__(self, name):

        # No anonymous superheroes on my watch! Every narcoleptic superhero gets
        # a name. Any name at all. SleepyMan. SlumberGirl. You get the idea.
        self.name = name

        # What have we accomplished today?
        self.kittens_rescued = 0

        # Initialize the state machine
        self.machine = Machine(model=self, states=NarcolepticSuperhero.states, initial='asleep')

        # Add some transitions. We could also define these using a static list of
        # dictionaries, as we did with states above, and then pass the list to
        # the Machine initializer as the transitions= argument.

        # At some point, every superhero must rise and shine.
        self.machine.add_transition(trigger='wake_up', source='asleep', dest='hanging out', after="some_func")

        # Superheroes need to keep in shape.
        self.machine.add_transition('work_out', 'hanging out', 'hungry')

        # Those calories won't replenish themselves!
        self.machine.add_transition('eat', 'hungry', 'hanging out')

        # Superheroes are always on call. ALWAYS. But they're not always
        # dressed in work-appropriate clothing.
        self.machine.add_transition('distress_call', '*', 'saving the world',
                         before='change_into_super_secret_costume')

        # When they get off work, they're all sweaty and disgusting. But before
        # they do anything else, they have to meticulously log their latest
        # escapades. Because the legal department says so.
        self.machine.add_transition('complete_mission', 'saving the world', 'sweaty',
                         after='update_journal')

        # Sweat is a disorder that can be remedied with water.
        # Unless you've had a particularly long day, in which case... bed time!
        self.machine.add_transition('clean_up', 'sweaty', 'asleep', conditions=['is_exhausted'])
        self.machine.add_transition('clean_up', 'sweaty', 'hanging out')

        # Our NarcolepticSuperhero can fall asleep at pretty much any time.
        self.machine.add_transition('nap', '*', 'asleep')

    def update_journal(self):
        """ Dear Diary, today I saved Mr. Whiskers. Again. """
        self.kittens_rescued += 1

    @property
    def is_exhausted(self):
        """ Basically a coin toss. """
        return random.random() < 0.5

    def change_into_super_secret_costume(self):
        print("Beauty, eh?")

    def some_func(self):
        print('Some function here.')
        

In [35]:
batman = NarcolepticSuperhero("Batman")
batman.state

'asleep'

In [36]:
batman.wake_up()
batman.state

Some function here.


'hanging out'

In [37]:
batman.nap()
batman.state

'asleep'

In [38]:
batman.wake_up()
batman.work_out()
batman.state

Some function here.


'hungry'

In [39]:
batman.distress_call()
batman.state

Beauty, eh?


'saving the world'

In [40]:
batman.complete_mission()
batman.state

'sweaty'

In [41]:
batman.clean_up()
batman.state

'asleep'

### Lab: Produce key-pair semantic synth-data by stage

In [44]:
import dspy
from dspy.functional import TypedPredictor
from pydantic import BaseModel, Field
from typing import List
from transitions import Machine
from dotenv import dotenv_values
from rich import print

secret = dotenv_values('../../.secret')
llm  = dspy.OpenAI(
    model='gpt-3.5-turbo',
    # model='gpt-4',
    # model='gpt-4o',
    api_key=secret['OPEN_AI_API_KEY'],
    max_tokens=4096
)

dspy.settings.configure(lm=llm)

# groq = dspy.GROQ(model='llama3-70b-8192', api_key=secret['GROQ_API_KEY'], max_tokens=2000)
# dspy.settings.configure(lm=groq, )


/Users/beltre.wilton/miniforge3/envs/tars_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
class CoTSignature(dspy.Signature):
    """Answer the question and give the reasoning for the same."""

    question = dspy.InputField(desc="question about something")
    answer = dspy.OutputField(desc="often between 1 and 5 words")

class CoTPipeline(dspy.Module):
    def __init__(self):
        super().__init__()

        self.signature = CoTSignature
        self.predictor = dspy.ChainOfThought(self.signature)

    def forward(self, question):
        result = self.predictor(question=question)
        print(result)
        return dspy.Prediction(
            answer=result.answer,
            reasoning=result.rationale,
        )
    

cot = CoTPipeline()
cot(question="What is the capital of Paris?")

Prediction(
    rationale='Answer: Paris\nReasoning: Paris is the capital of France.',
    answer='Paris'
)

Prediction(
    answer='Paris',
    reasoning='Answer: Paris\nReasoning: Paris is the capital of France.'
)